In [3]:
import os
import pandas as pd
from datetime import datetime

In [4]:
# 路径设置
base_dir = "users_timeXYZ/users/"  # 加速度计数据的主目录
TestActivities = "TestActivities.csv"  # 活动标签文件

# 初始化空的 DataFrame 用于存储所有加速度数据
all_accel_data = pd.DataFrame()

# 遍历所有子文件夹
for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):  # 确保是文件夹
        # 遍历文件夹中的所有 CSV 文件
        for file in os.listdir(folder_path):
            if file.endswith(".csv"):  # 确保是 CSV 文件
                file_path = os.path.join(folder_path, file)
                # 读取 CSV 文件并手动指定列名
                data = pd.read_csv(
                    file_path, 
                    header=None,  # 因为文件没有列名
                    names=["RandomID", "Timestamp", "Accel_X", "Accel_Y", "Accel_Z"]  # 手动指定列名
                )
                
                # 1. 去除加速度数据中的缺失值
                data = data.dropna(subset=["Timestamp", "Accel_X", "Accel_Y", "Accel_Z"])
                
                # 2. 选择有用的列（忽略 RandomID 列）
                data = data[["Timestamp", "Accel_X", "Accel_Y", "Accel_Z"]]
                
                # 3. 将时间戳转换为标准格式，去除无效时间戳
                data["Timestamp"] = pd.to_datetime(
                    data["Timestamp"], 
                    format="%Y-%m-%dT%H:%M:%S.%f%z",  # 解析时间戳样式
                    errors="coerce"  # 无法解析的时间戳设置为 NaT
                )
                
                # 4. 去除转换后无效的时间戳数据
                data = data.dropna(subset=["Timestamp"])
                
                # 5. 创建 'Minute' 列，提取时间戳的分钟部分
                data["Minute"] = data["Timestamp"].dt.floor("min")
                
                # 6. 添加文件来源信息
                data["SourceFile"] = file
                
                # 7. 只保留有效数据
                if len(data) > 0:  # 确保还有数据
                    all_accel_data = pd.concat([all_accel_data, data], ignore_index=True)

print(f"处理完成，总数据行数: {len(all_accel_data)}")
print(f"其中有效时间戳数量: {all_accel_data['Timestamp'].notna().sum()}")
print(f"有效Minute数量: {all_accel_data['Minute'].notna().sum()}")

处理完成，总数据行数: 2045326
其中有效时间戳数量: 2045326
有效Minute数量: 2045326


In [5]:
# 先将 Timestamp 转成 datetime 并转换到UTC
all_accel_data["Timestamp"] = pd.to_datetime(
    all_accel_data["Timestamp"],
    format="%Y-%m-%dT%H:%M:%S.%f%z",  # 解析带时区的时间戳
    utc=True  # 转换到UTC
)

# 创建Minute列（向下取整到分钟）
all_accel_data["Minute"] = all_accel_data["Timestamp"].dt.floor("min")

# 按时间戳排序
all_accel_data = all_accel_data.sort_values(by="Timestamp").reset_index(drop=True)

# 保存预览数据
all_accel_data.to_csv("all_accel_data_preview.csv", index=False)

# 打印一些信息来验证
print("时间戳范围（UTC）：")
print(f"最早时间：{all_accel_data['Timestamp'].min()}")
print(f"最晚时间：{all_accel_data['Timestamp'].max()}")
print(f"\n总数据行数: {len(all_accel_data)}")

时间戳范围（UTC）：
最早时间：2024-09-01 21:25:07.752000+00:00
最晚时间：2024-09-12 20:19:47.443000+00:00

总数据行数: 2045326


In [6]:
print(all_accel_data["Minute"].dtype)  # 应该是 datetime64[ns]
print(all_accel_data["Timestamp"].dtype)

datetime64[ns, UTC]
datetime64[ns, UTC]


In [7]:
# 加载活动标签文件
label_data = pd.read_csv(TestActivities)

# 处理标签数据的时间 (2024/9/2 6:16:00 格式)
time_columns = ["Started", "Finished", "Updated"]
for col in time_columns:
    if col in label_data.columns:
        # 先转换为datetime（此时是UTC+9本地时间）
        local_time = pd.to_datetime(label_data[col], format='%Y/%m/%d %H:%M')
        # 添加UTC+9时区信息并转换到UTC
        label_data[col] = (
            local_time.dt.tz_localize('+0900')  # 标记为UTC+9时区
            .dt.tz_convert('UTC')               # 转换到UTC
        )
# 基于 'Started' 列进行匹配，保留加速度数据的毫秒级时间戳
merged_test_data = pd.merge(
    all_accel_data[all_accel_data["Minute"].notna()],  # 只使用有效的时间戳
    label_data[label_data["Started"].notna()],         # 只使用有效的标签时间
    how="right",
    left_on="Minute",
    right_on="Started"
)
# 保存最终合并后的数据集
merged_test_data.to_csv("final_merged_test_data.csv", index=False)

print("数据处理完成，最终数据已保存为 'final_merged_test_data.csv'")

# 4. 打印一些统计信息
print(f"清理后的数据行数: {len(merged_test_data)}")

数据处理完成，最终数据已保存为 'final_merged_test_data.csv'
清理后的数据行数: 428404


In [10]:
###############################################################################
# (2) 简单数据清洗与标准化
###############################################################################
# 1) 去除加速度中的缺失值（Accel_X, Accel_Y, Accel_Z）
merged_test_data = merged_test_data.dropna(subset=["Accel_X", "Accel_Y", "Accel_Z"])

# 2) 过滤极端异常 (假定绝对值 > 50 属于异常)
merged_test_data = merged_test_data[
    (merged_test_data["Accel_X"].abs() <= 50) &
    (merged_test_data["Accel_Y"].abs() <= 50) &
    (merged_test_data["Accel_Z"].abs() <= 50)
]
# # # 5) 使用 StandardScaler 对加速度列做标准化：
# from sklearn.preprocessing import StandardScaler

# # 如果你想按每个 Subject 单独标准化，则 groupby("Subject") 处理：
# # 如果不需要分主体，也可以整体标准化
# df_list = []
# for subj, grp in merged_test_data.groupby("Subject"):
#     scaler = StandardScaler()
#     grp[["Accel_X", "Accel_Y", "Accel_Z"]] = scaler.fit_transform(
#         grp[["Accel_X", "Accel_Y", "Accel_Z"]]
#     )
#     df_list.append(grp)

# merged_test_data = pd.concat(df_list, ignore_index=True)
merged_test_data = merged_test_data.sort_values(by="Timestamp").reset_index(drop=True)
#删除timestamp重复的行
merged_test_data=merged_test_data.drop_duplicates(subset=['Timestamp'],keep='first')

print("完成清洗后剩余行数:", len(merged_test_data))

# 6) 将清洗后的数据写回新的 CSV 文件
merged_test_data.to_csv("cleaned_test_data.csv", index=False)
print("已将清洗+标准化后的数据保存至: cleaned_test_data.csv")

完成清洗后剩余行数: 196947
已将清洗+标准化后的数据保存至: cleaned_test_data.csv


In [12]:
import os
import pandas as pd

# 读取数据
merged_test_data = pd.read_csv('cleaned_test_data.csv')

# 确保目标文件夹存在
os.makedirs('minute_data', exist_ok=True)

# 遍历每个唯一的 Minute 值
for minute in merged_test_data['Minute'].unique():
    # 筛选并排序数据（保持原始列顺序）
    minute_data = merged_test_data[merged_test_data['Minute'] == minute]\
                    .sort_values(by="Timestamp")\
                    .iloc[:, :4]  # 修正这里：改为取前4列（索引0-3）

    # 构造安全文件名
    safe_minute = str(minute).translate(str.maketrans(":+ ", "---"))
    file_path = os.path.join('minute_data', f"{safe_minute}.csv")
    
    # 保存数据（保持原始列顺序）
    minute_data.to_csv(file_path, index=False, header=False)
    
    # 调试输出
    print(f"Minute: {minute} | Shape: {minute_data.shape}")
    print("Columns order preserved:", minute_data.columns.tolist()[:4])  # 验证列顺序
    print('=' * 20)

Minute: 2024-09-08 22:47:00+00:00 | Shape: (1047, 4)
Columns order preserved: ['Timestamp', 'Accel_X', 'Accel_Y', 'Accel_Z']
Minute: 2024-09-08 22:48:00+00:00 | Shape: (2153, 4)
Columns order preserved: ['Timestamp', 'Accel_X', 'Accel_Y', 'Accel_Z']
Minute: 2024-09-10 20:14:00+00:00 | Shape: (3144, 4)
Columns order preserved: ['Timestamp', 'Accel_X', 'Accel_Y', 'Accel_Z']
Minute: 2024-09-10 20:15:00+00:00 | Shape: (3139, 4)
Columns order preserved: ['Timestamp', 'Accel_X', 'Accel_Y', 'Accel_Z']
Minute: 2024-09-10 20:16:00+00:00 | Shape: (3145, 4)
Columns order preserved: ['Timestamp', 'Accel_X', 'Accel_Y', 'Accel_Z']
Minute: 2024-09-10 20:17:00+00:00 | Shape: (3142, 4)
Columns order preserved: ['Timestamp', 'Accel_X', 'Accel_Y', 'Accel_Z']
Minute: 2024-09-10 20:18:00+00:00 | Shape: (3142, 4)
Columns order preserved: ['Timestamp', 'Accel_X', 'Accel_Y', 'Accel_Z']
Minute: 2024-09-10 20:19:00+00:00 | Shape: (3145, 4)
Columns order preserved: ['Timestamp', 'Accel_X', 'Accel_Y', 'Accel_Z']
